In [14]:
import tensorflow as tf
import tensorflow.keras as keras

In [30]:
def upscale_block(inpx, upsamp_val,prevx,t_filt,conv1=False):
    if conv1:
        xa = tf.image.resize(inpx,upsamp_val)
        xa = Conv2D(512,1,padding="same")(xa)
        xa = BatchNormalization(axis=-1)(xa)
        xa = Activation("relu")(xa)
        xb = Conv2D(512,1,padding="same")(prevx)
        xb = BatchNormalization(axis=-1)(xb)
        xb = Activation("relu")(xb)
        xconc = concatenate([xa,xb],axis=3)
        xaxb = Conv2D(t_filt,3,padding="same")(xconc)
        xaxb = BatchNormalization(axis=-1)(xaxb)
        xaxb = Activation("relu")(xaxb)
    else:
        xa = tf.image.resize(inpx,upsamp_val)
        xconc = concatenate([xa,prevx],axis=3)
        xaxb = Conv2D(t_filt,3,padding="same")(xconc)
        xaxb = BatchNormalization(axis=-1)(xaxb)
        xaxb = Activation("relu")(xaxb)
    return xaxb

def UNET(classes):
    base_model = tf.keras.applications.ResNet50(include_top=False,input_shape=(350,350,3))
    for layer in base_model.layers:
        layer.traiable = True
    a1 = base_model.get_layer("activation").output  
    f1 = base_model.get_layer("activation_2").output   
    f2 = base_model.get_layer("activation_20").output   
    f3 = base_model.get_layer("activation_31").output 
    f4 = base_model.get_layer("activation_36").output  
    f5 = base_model.get_layer("activation_42").output  
    
    u5 = upscale_block(f5,(22,22),f4,512,conv1=True)
    u4 = upscale_block(u5,(22,22),f3,256)
    u3 = upscale_block(u4,(44,44),f2,128)
    u2 = upscale_block(u3,(88,88),f1,64)
    u1 = upscale_block(u2,(175,175),a1,64)
    
    u1 = tf.image.resize(u1,(350,350))
    au1 = Conv2D(32,3,padding="same")(u1)
    au1 = BatchNormalization(axis=3)(au1)
    au1 = PReLU(shared_axes=[1,2])(au1)
    
    au1 = Conv2D(classes,1,padding="same")(au1)
    au1 = Activation("softmax")(au1)
    build = Model(inputs=base_model.input,outputs=au1)
    return build

In [31]:
unet = UNET(3)

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/resnet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5: 403 -- Forbidden